<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 作業用ディレクトリへ移動
# システムコマンドは基本「！」をつけるが、ディレクトリ移動は「％」らしい
!mkdir chap7
%cd ./chap7


/content/chap7


In [2]:
# いつものライブラリに加えて、ファインチューニングと性能評価を行うPyTorch Lightningも使用
# バージョンが教科書のだと古いようなので、模範解答（https://github.com/stockmarkteam/bert-book/blob/master/Chapter6.ipynb）にバージョンを合わせている
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

# またNo module named ‘torchtext.legacy’エラーが出たため
# https://masaki-note.com/2022/05/27/torchtext_legacy_error/
# に従い以下を実行してない
# !pip install --upgrade torchtext==0.9.1
# 2行目のバージョン調整で対処

# いろいろなライブラリたち
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl

# 日本語事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 20.6 MB/s 
     |████████████████████████████████| 486 kB 33.0 MB/s 
     |████████████████████████████████| 13.4 MB 60.8 MB/s 
     |████████████████████████████████| 582 kB 9.6 MB/s 
     |████████████████████████████████| 880 kB 46.9 MB/s 
     |████████████████████████████████| 120 kB 44.9 MB/s 
     |████████████████████████████████| 6.6 MB 51.4 MB/s 
     |████████████████████████████████| 419 kB 49.1 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=0d8aa5048edb1454911ee54a8448d412e2f56d68092a316bb43bca50a50e772a
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d721e3e4bb3dac40d0cb15a3c32f84bf083c8963254b49f02f2f1251bfade78b
  Stored in direc

In [3]:
'''
マルチラベル分類：一つの文章に対して、複数のカテゴリを割り当てる分類方法
シングルラベル分類：一つの文章に対して、一つのカテゴリを割り当てる分類方法


## マルチラベル分類
文章の属すカテゴリーをMulti-hotベクトルと呼ぶ形式で表現する
  サイズがカテゴリ数と同じ
  文章が属しているカテゴリに対応する要素に１、それ以外に０が入る

  文章が属しているカテゴリ　＝　分類スコアが正の値のカテゴリ
  文章が属していないカテゴリ　＝　分類スコアが負の値のカテゴリ
  とする（０はどっちに入るのか分からない・・・）


## BertForSequenceClassificationMultiLabel
マルチラベル分類を行うクラス（あとで定義します）

入力：文章（複数バッチ、符号化済み）
出力：文章ごとのMulti-hotベクトル


処理
１。入力をBertModelにつっこみ、出力（各トークンに対する分散表現）を得る
２。得られたベクトルを[PAD]（系列長長さ合わせのために挿入されたトークン）に対応するものを除いて平均化する
３。平均化したものを線形変換する
４。上記で得られた入力の１文ごとに対するMulti-hotベクトルが出力

BertForSequenceClassification(シングルラベル分類)との違い
  損失関数がバイナリクロスエントロピー（シングルの時はただのクロスエントロピーだった）
    cf : https://yaakublog.com/crossentropy_binarycrossentropy
    
    分類スコアに対してシグモイド関数を適用した値を計算する
      シグモイド関数の出力：そのカテゴリに文章が属する確率→50%以上の時を正とみなす
    そして、そうやって求めた予測確率と、実際に文章がそのカテゴリに属しているかどうかとの間のバイナリクロスエントロピーを求める

  

'''

'\nマルチラベル分類：一つの文章に対して、複数のカテゴリを割り当てる分類方法\nシングルラベル分類：一つの文章に対して、一つのカテゴリを割り当てる分類方法\n\n\n## マルチラベル分類\n文章の属すカテゴリーをMulti-hotベクトルと呼ぶ形式で表現する\n  サイズがカテゴリ数と同じ\n  文章が属しているカテゴリに対応する要素に１、それ以外に０が入る\n\n  文章が属しているカテゴリ\u3000＝\u3000分類スコアが正の値のカテゴリ\n  文章が属していないカテゴリ\u3000＝\u3000分類スコアが負の値のカテゴリ\n  とする（０はどっちに入るのか分からない・・・）\n\n\n## BertForSequenceClassificationMultiLabel\nマルチラベル分類を行うクラス（あとで定義します）\n\n入力：文章（複数バッチ、符号化済み）\n出力：文章ごとのMulti-hotベクトル\n\n\n処理\n１。入力をBertModelにつっこみ、出力（各トークンに対する分散表現）を得る\n２。得られたベクトルを[PAD]（系列長長さ合わせのために挿入されたトークン）に対応するものを除いて平均化する\n３。平均化したものを線形変換する\n４。上記で得られた入力の１文ごとに対するMulti-hotベクトルが出力\n\nBertForSequenceClassification(シングルラベル分類)との違い\n  損失関数がバイナリクロスエントロピー（シングルの時はただのクロスエントロピーだった）\n    cf : https://yaakublog.com/crossentropy_binarycrossentropy\n    \n    分類スコアに対してシグモイド関数を適用した値を計算する\n      シグモイド関数の出力：そのカテゴリに文章が属する確率→50%以上の時を正とみなす\n    そして、そうやって求めた予測確率と、実際に文章がそのカテゴリに属しているかどうかとの間のバイナリクロスエントロピーを求める\n\n  \n\n'

In [4]:
# 実装
# torch.nn　の nn は多分 neural network
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html

class BertForSequenceClassificationMultiLabel(torch.nn.Module):

    def __init__(self, model_name, num_labels):
        super().__init__()

        # BertModelをロード
        self.bert = BertModel.from_pretrained(MODEL_NAME)

        # 線形変換の初期化
        # https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
        # https://free.kikagaku.ai/tutorial/basic_of_deep_learning/learn/pytorch_beginners
        # 線形変換：w1x1 + w2x2 + ......  みたいな。重みは学習する
        self.linear = torch.nn.Linear (
            self.bert.config.hidden_size, num_labels
        )

  
    # forward : データを入力し、BERTの最終層の出力を得る
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None
    ):
        # データ(input_ids, attention_mask([PAD]の位置を表す), token_type_ids(何文目かを表す))をBERTにを入力
        # attention_maskって、[PAD]に対応するところは0でそれ以外は1のベクトル
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # bert_outputのうち、最終層の出力だけ取得
        last_hidden_state = bert_output.last_hidden_state

        # [PAD]以外のトークンで隠れ状態の平均を取る
        # \ (macだとoption + ¥) で改行できるっぽい
        # unsqueeze(num) : torch.Tensorの次元をnumだけ上げてくれる
        # https://pytorch.org/docs/stable/generated/torch.unsqueeze.html
        # sum : https://zanote.net/python/pytorch-torch-sum/
        averaged_hidden_state = \
            (last_hidden_state*attention_mask.unsqueeze(-1)).sum(1) \
            / attention_mask.sum(1, keepdim=True)

        #　上記をさらに線形変換
        scores = self.linear(averaged_hidden_state)

        # 出力形式をいつものBertModelと同様になるように整える
        output = {'logits' : scores}

        # labelsが入力に含まれていたら、損失を計算して出力する
        # BCE : バイナリクロスエントロピー
        # torch.nn.BCEWithLogitsLoss : シグモイド関数で変換した値のバイナリクロスエントロピを計算する関数
        # https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html
        # https://runebook.dev/ja/docs/pytorch/generated/torch.nn.bcewithlogitsloss
        if labels is not None:
            loss = torch.nn.BCEWithLogitsLoss() (scores, labels.float())
            output['loss'] = loss

        # 属性でアクセスできるようにする
        # ちょっとよく分からん・・・
        output = type('bert_output', (object, ), output)

        return output




In [5]:
# では、定義したモデルを使用してマルチラベル分類を行いましょう

# トークナイザ
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# バートモデル定義
bert_scml = BertForSequenceClassificationMultiLabel(
    model_name=MODEL_NAME, num_labels=2
)

# GPUへ
bert_scml = bert_scml.cuda()

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# まず学習なしで、結果を出力してみましょうね
# 分類対象のテキストリスト
text_list = [
    '今日の仕事はうまくいったが、体調があまり良くない。',
    '昨日は楽しかった。'
]


# ラベルのリスト
# [negativeに属するなら1, positiveに属するなら1]
labels_list = [
    [1, 1],
    [0, 1]
]


# データの符号化
encoding = tokenizer(
    text_list, 
    padding = 'longest',
    return_tensors='pt'
)

# 以下の結果から、tokenizerを直接関数呼び出しする（上記の）スタイルのinput_idsは
# encodingメソッドの出力たちである
print(encoding)
print(tokenizer.encode(text_list[0], return_tensors='pt'))
print(tokenizer.tokenize(text_list[0]))

encoding = { k : v.cuda() for k, v in encoding.items() }
labels = torch.tensor(labels_list).cuda()


# BERTにデータを入力
with torch.no_grad():
    # bert_scmlに引数を渡すと、call関数経由でfowardが呼び出されるっぽい
    # https://teratail.com/questions/221589
    output = bert_scml(**encoding)
scores = output.logits


# スコアが正ならば、そのカテゴリを選択する
# つまり、そのカテゴリに含まれていると判定
labels_predicted = (scores > 0).int()


# 精度の計算
# all は多分これ
# https://pytorch.org/docs/stable/generated/torch.all.html
# item()
# PyTorchのTensorの要素の値を取得: item()
num_correct = (labels_predicted == labels).all(-1).sum().item()
accuracy = num_correct / labels.size(0)



print(labels)
print(labels_predicted)
print(accuracy)

{'input_ids': tensor([[    2,  3246,     5,  2198,     9,  9291,  1281,    10,    14,     6,
         10522,    14,  2482,  5735,    80,     8,     3],
        [    2, 10271, 28486,     9,  4613,   187,    10,     8,     3,     0,
             0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}
tensor([[    2,  3246,     5,  2198,     9,  9291,  1281,    10,    14,     6,
         10522,    14,  2482,  5735,    80,     8,     3]])
['今日', 'の', '仕事', 'は', 'うまく', 'いっ', 'た', 'が', '、', '体調', 'が', 'あまり', '良く', 'ない', '。']
tensor([[1, 1],
        [0, 1]], device='cuda:0')
tensor([[0, 0],
        [0, 1]], device='cuda:0', dtype=torch.int32)
0.5


In [7]:
# 学習時にはencodingの属性に['labels']を追加せよ

encoding['labels'] = torch.tensor(labels_list)
encoding = { k : v.cuda() for k, v in encoding.items() }

output = bert_scml(**encoding)
loss = output.loss # 損失

In [8]:
# さぁて実際学習するぞ
# データセット：chABSA-dataset
# ここではネガティブ、ポジティブ、ニュートラル（中立）の３カテゴリに、単語ごとの分類を行っている

# データダウンロード
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
!unzip chABSA-dataset.zip


--2022-09-05 12:25:44--  https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip
Resolving s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)... 52.219.197.4
Connecting to s3-ap-northeast-1.amazonaws.com (s3-ap-northeast-1.amazonaws.com)|52.219.197.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 722777 (706K) [application/zip]
Saving to: ‘chABSA-dataset.zip’

chABSA-dataset.zip  100%[===================>] 705.84K  1.77MB/s    in 0.4s    

2022-09-05 12:25:45 (1.77 MB/s) - ‘chABSA-dataset.zip’ saved [722777/722777]

Archive:  chABSA-dataset.zip
   creating: chABSA-dataset/
  inflating: chABSA-dataset/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/chABSA-dataset/
  inflating: __MACOSX/chABSA-dataset/._.DS_Store  
 extracting: chABSA-dataset/.gitkeep  
  inflating: chABSA-dataset/e00008_ann.json  
  inflating: chABSA-dataset/e00017_ann.json  
  inflating: chABSA-dataset/e00024_ann.json  
  inflating: 

In [9]:
# ダウンロードしたデータはJSON形式で保存
# porality の項目がカテゴリを表す
import json
data = json.load(open('chABSA-dataset/e00030_ann.json'))
print(data['sentences'][0])

{'sentence_id': 0, 'sentence': '当期におけるわが国経済は、景気は緩やかな回復基調が続き、設備投資の持ち直し等を背景に企業収益は改善しているものの、海外では、資源国等を中心に不透明な状況が続き、為替が急激に変動するなど、依然として先行きが見通せない状況で推移した', 'opinions': [{'target': 'わが国経済', 'category': 'NULL#general', 'polarity': 'neutral', 'from': 6, 'to': 11}, {'target': '景気', 'category': 'NULL#general', 'polarity': 'positive', 'from': 13, 'to': 15}, {'target': '設備投資', 'category': 'NULL#general', 'polarity': 'positive', 'from': 28, 'to': 32}, {'target': '企業収益', 'category': 'NULL#general', 'polarity': 'positive', 'from': 42, 'to': 46}, {'target': '資源国等', 'category': 'NULL#general', 'polarity': 'neutral', 'from': 62, 'to': 66}, {'target': '為替', 'category': 'NULL#general', 'polarity': 'negative', 'from': 80, 'to': 82}]}


In [12]:
# 上記のデータセットを整形
# 特に、カテゴリをMulti-hot ベクトル[negative, neutral, positive]で表す

category_id = {'negative': 0, 'neutral': 1, 'positive': 2}

# 整形後データセットを入れる箱
dataset = []

for file in glob.glob('chABSA-dataset/*json'):
    # データを開く
    data = json.load(open(file))

    # 各データから文章（text）を抜き出し、ラベル('labels')を作成
    for sentence in data['sentences']:
        # 文章を１文丸ごと取得
        text = sentence['sentence']

        # ラベル初期化
        labels = [0, 0, 0]

        # opinions(単語ごとの分類などが入った属性)を展開
        for opinion in sentence['opinions']:
            # category_idでpolarityに対応する数字を取得し、その位置に1を入れる
            labels[category_id[opinion['polarity']]] = 1
        
        # 整形後データ（１文）
        sample = {'text': text, 'labels': labels}
        # 整形後データセットに追加
        dataset.append(sample)





In [13]:
# 整形後データ
print(dataset[0])

{'text': '平成28年度の沖縄県経済は、個人消費や観光が堅調で、建設関連も公共投資が底堅く推移していることなどにより、全体として拡大した', 'labels': [0, 0, 1]}


In [15]:
# さてここからは、ファインチューニングをおこないますよ

# データセット分割
# 学習６０％、検証２０％、テスト２０％

# データ形式をDataLoaderに入力できるように整える
# つまり、tokenizerのインスタンスを関数として呼び出して１文丸ごとエンコーディングしたものに、labels属性を追加し、dict型（valueはtensor）に整形する

# 整形後のデータの入れ物
dataset_for_loader = []
# トークン数は最大128まで
max_length = 128

for sample in dataset:
    text = sample['text']
    labels = sample['labels']

    encoding = tokenizer(
        text,
        max_length=max_length,
        padding = 'max_length',
        truncation = True
    )

    encoding['labels'] = labels
    encoding = { k : torch.tensor(v) for k, v in encoding.items() }

    dataset_for_loader.append(encoding)



In [16]:
# 分割
# まずはdataset_for_loaderをランダムに並び替える
random.shuffle(dataset_for_loader)

# データセットの総数n
n = len(dataset_for_loader)

# 訓練データセットの大きさ
n_train = int (n*0.6)
# 検証、テストデータセットの大きさ
n_val = int(n*0.2)

# 各種データセットの定義
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train: (n_train+n_val)]
dataset_test = dataset_for_loader[(n_train+n_val):]

# データローダの作成
dataloader_train = DataLoader(
    dataset_train, batch_size = 32, shuffle=True
)

dataloader_val = DataLoader(
    dataset_val, batch_size=256
)

dataloader_test = DataLoader(
    dataset_test, batch_size = 256
)



In [34]:
# PyTorch Lightningの定義
class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters()
        self.bert_scml = BertForSequenceClassificationMultiLabel(
            model_name=model_name, num_labels=num_labels
        )


    def training_step(self, batch, batch_idx):
        # lossを返す
        output = self.bert_scml(
            **batch
        )

        loss = output.loss
        self.log('train_loss', loss)
        return loss

    
    def validation_step(self, batch, batch_idx):
        # lossを求める
        output = self.bert_scml(**batch)
        val_loss = output.loss
        self.log('val_loss', loss)

    def test_step(self, batch, batch_idx):
        # accuracyを求める
        labels = batch.pop('labels')
        output = self.bert_scml(**batch)

        scores = output.logits
        labels_predicted = (scores > 0).int()

        num_correct = (labels_predicted == labels).all(-1).sum().item()
        accuracy = num_correct / scores.size(0)

        self.log('accuracy', accuracy)


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [21]:
# チェックポイント生成
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor = 'val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/'
)

In [22]:
# 訓練モジュール
trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    callbacks=[checkpoint]
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [35]:
# さて、ファインチューニングを実際に行え
model = BertForSequenceClassificationMultiLabel_pl(
    model_name=MODEL_NAME,
    num_labels=3,
    lr=1e-5
)

trainer.fit(model, dataloader_train, dataloader_val)
test = trainer.test(dataloaders=dataloader_test)
print(test[0])
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1445: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/chap7/model/epoch=0-step=115.ckpt
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/chap7/model/epoch=0-step=115.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.8767346739768982
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'accuracy': 0.8767346739768982}
Accuracy: 0.88


In [36]:
# 訓練したモデルで実際に分類を行ってみよう

# 入力文章
text_list = [
    "今期は売上が順調に推移したが、株価は低迷の一途を辿っている。",
    "去年から黒字が減少した。",
    "今日の飲み会は楽しかった。"
]

# モデルのロード
best_model_path=checkpoint.best_model_path
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(best_model_path)

print(best_model_path)
bert_scml = model.bert_scml.cuda()

# データの符号化
encoding = tokenizer(
    text_list,
    padding='longest',
    return_tensors='pt'
)
encoding = { k : v.cuda() for k, v in encoding.items() }

# BERTへデータを入力して分類スコアを得る
output = bert_scml(**encoding)
scores = output.logits
labels_predicted = (scores > 0).int().cpu().numpy().tolist()

# 結果を表示
for text, label in zip(text_list, labels_predicted):
    print('--')
    print(f'入力：{text}')
    print(f'出力：{label}')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/content/chap7/model/epoch=0-step=115.ckpt
--
入力：今期は売上が順調に推移したが、株価は低迷の一途を辿っている。
出力：[1, 0, 1]
--
入力：去年から黒字が減少した。
出力：[1, 0, 0]
--
入力：今日の飲み会は楽しかった。
出力：[0, 0, 0]
